In [1]:
import cv2
import numpy as np
import pandas as pd

In [2]:
def get_video_properties(video_path):
    """
    Retrieves video properties such as total frames, frame height, and frame width.

    :param video_path: Path to the video file.
    :return: A tuple containing the total number of frames, frame height, and frame width.
    """
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError("Error opening video file: {}".format(video_path))

    # Get total number of frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Get frame height and width
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Release the video capture object
    cap.release()

    return total_frames, frame_height, frame_width, fps

def extract_key_frames(dataframe, threshold, video_path, n_sec_side=1):
    """
    Divide video into 1-second segments and extract all frames from the segment containing at least one key frame
    and an additional n_sec_side seconds (segments) on each side of such a segment. The output is a dictionary
    where each key represents the index of a segment group and the value is a list of frame IDs within that group.

    :param dataframe: DataFrame containing the frame data of potential keyframes.
    :param threshold: The frame_c_score threshold to identify key frames.
    :param video_path: Path to the video file for frame count extraction.
    :param n_sec_side: Number of seconds (segments) to include on each side of a segment containing at least one key frame.
    :return: A dictionary of unique frame IDs from segments closely surrounding those containing at least one key frame,
             grouped by segment group index.
    """

    total_frames, _, _, fps = get_video_properties(video_path)
    segment_duration = 1  # Duration of each segment in seconds
    frames_per_segment = fps * segment_duration

    qualifying_segments = {}

    # Iterate over segments to identify qualifying segments
    for segment_start_frame in range(0, total_frames, frames_per_segment):
        segment_end_frame = min(segment_start_frame + frames_per_segment, total_frames)

        # Check if the segment contains at least one key frame
        if not dataframe[(dataframe['frame_id'] >= segment_start_frame) &
                         (dataframe['frame_id'] < segment_end_frame) &
                         (dataframe['frame_c_score'] >= threshold)].empty:

            # Calculate the extended start and end frame indices for the group
            extended_start_frame = max(0, segment_start_frame - (frames_per_segment * n_sec_side))
            extended_end_frame = min(total_frames, segment_end_frame + (frames_per_segment * n_sec_side))

            # Calculate the group index based on the current segment start frame
            group_index = segment_start_frame // frames_per_segment

            # Adjust group index for the surrounding segments
            group_index = group_index - n_sec_side if (group_index - n_sec_side) > 0 else 0

            # Add all frames in this extended range to the corresponding group in the dictionary
            if group_index not in qualifying_segments:
                qualifying_segments[group_index] = list(range(extended_start_frame, extended_end_frame))
            else:
                # Update the group if it already exists to ensure we're not duplicating segments
                new_frames = set(qualifying_segments[group_index])
                new_frames.update(range(extended_start_frame, extended_end_frame))
                qualifying_segments[group_index] = sorted(new_frames)

    # Ensure the frame IDs in each group are unique and sorted
    for key in qualifying_segments:
        qualifying_segments[key] = sorted(set(qualifying_segments[key]))

    return qualifying_segments

# def consolidate_frame_ids(segment_groups):
#     """
#     Consolidate all frame IDs from grouped segments into a single, sorted list with duplicates removed.

#     :param segment_groups: A dictionary where each key is a group index and each value is a list of frame IDs.
#     :return: A sorted list of unique frame IDs across all groups.
#     """

#     all_frames = set()

#     for frame_list in segment_groups.values():
#         all_frames.update(frame_list)

#     # Convert the set to a sorted list
#     consolidated_frames = sorted(all_frames)

#     return consolidated_frames

In [5]:
def merge_segments_with_original_indexes(segments):
    # Sort segments by their start frame, but keep original indexes
    sorted_segments = sorted(((idx, frames[0], frames[-1]) for idx, frames in segments.items()), key=lambda x: x[1])

    merged = []
    for idx, current_start, current_end in sorted_segments:
        # If merged is not empty and the current segment overlaps or directly follows the last one in merged
        if merged and current_start <= merged[-1][2] + 1:
            # Update the last segment's end to the max of both ends, keeping the smallest index
            merged[-1] = (merged[-1][0], merged[-1][1], max(merged[-1][2], current_end))
        else:
            # Add the current segment as a new entry (index, start, end)
            merged.append((idx, current_start, current_end))

    # Convert back to the original format with original indexes as keys
    merged_segments = {idx: list(range(start, end + 1)) for idx, start, end in merged}
    return merged_segments

In [6]:
dfk = pd.read_csv("/content/drive/MyDrive/Project/Object_tracking/peaks_output_by_frame.csv")
df_to_plot = pd.read_pickle('/content/drive/MyDrive/Project/Object_tracking/scan1_joints_and_masks_to_plot_updated.pkl')
video_path = "/content/drive/MyDrive/Project/Object_tracking/scan_video1.avi"

In [8]:
extracted_frames = extract_key_frames(dfk, threshold=10, video_path=video_path, n_sec_side=1)
print(extracted_frames)
print(len(extracted_frames))

merged_frame_segments = merge_segments_with_original_indexes(extracted_frames)
print(merged_frame_segments)
print(len(merged_frame_segments))

{1: [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], 2: [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124], 12: [300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371,

In [85]:
reba_threshold = 10

df_info = pd.DataFrame(columns=['start_frame', 'end_frame', 'start_sec', 'c_score', 'leg_score','trunk_score','neck_score','wrist_score','lower_arm_score','upper_arm_score', 'inter_w_hand', 'inter_w_foot'])

for i in range(len(merged_frame_segments)):
  start_sec = list(merged_frame_segments.keys())[i]
  frames = list(merged_frame_segments.values())[i]
  start_frame = frames[0]
  end_frame = frames[-1]
  df_info.loc[i, 'start_frame'] = start_frame
  df_info.loc[i, 'end_frame'] = end_frame
  df_info.loc[i, 'start_sec'] = start_sec
  c_scores = []
  leg_scores = []
  trunk_scores = []
  neck_scores = []
  wrist_scores = []
  lower_arm_scores = []
  upper_arm_scores = []
  inter_w_hand = []
  inter_w_foot = []
  for frame_id in frames:
    if frame_id in set(dfk['frame_id'].tolist()) and dfk[dfk['frame_id']== frame_id]['frame_c_score'].tolist()[0] >= reba_threshold:
      c_score, leg_score, trunk_score, neck_score, wrist_score, lower_arm_score, upper_arm_score = dfk[dfk['frame_id']== frame_id][['frame_c_score','leg_score','trunk_score','neck_score','wrist_score','lower_arm_score','upper_arm_score']].iloc[0].tolist()
      c_scores.append(c_score)
      leg_scores.append(leg_score)
      trunk_scores.append(trunk_score)
      neck_scores.append(neck_score)
      wrist_scores.append(wrist_score)
      lower_arm_scores.append(lower_arm_score)
      upper_arm_scores.append(upper_arm_score)
      interactions = df_to_plot[df_to_plot['frame_id']== frame_id]['interactions'].tolist()[0]
      for label in range(len(interactions)):
        label_interaction = interactions[label]
        for obj_inter in label_interaction:
          if obj_inter[3] is not None or obj_inter[6] is not None:
            inter_w_hand.append(label)
          if obj_inter[9] is not None or obj_inter[12] is not None:
            inter_w_foot.append(label)
  df_info.loc[i,'c_score'] = np.max(c_scores)
  df_info.loc[i,'leg_score'] = np.max(leg_scores)
  df_info.loc[i,'trunk_score'] = np.max(trunk_scores)
  df_info.loc[i,'neck_score'] = np.max(neck_scores)
  df_info.loc[i,'wrist_score'] = np.max(wrist_scores)
  df_info.loc[i,'lower_arm_score'] = np.max(lower_arm_scores)
  df_info.loc[i,'upper_arm_score'] = np.max(upper_arm_scores)
  if len(inter_w_hand) > 0:
   df_info.at[i,'inter_w_hand'] = sorted(set(inter_w_hand))
  if len(inter_w_foot) > 0:
    df_info.at[i,'inter_w_foot'] = sorted(set(inter_w_foot))

df_info

,start_frame,end_frame,start_sec,c_score,leg_score,trunk_score,neck_score,wrist_score,lower_arm_score,upper_arm_score,inter_w_hand,inter_w_foot
0,25,124,1,10,3,4,2,2,2,5,[1],NaN
1,300,374,12,10,3,4,2,2,2,5,NaN,NaN
2,425,724,17,10,3,4,2,2,2,5,[0],NaN
3,1100,1174,44,10,3,4,2,2,2,5,NaN,[1]
4,1225,1299,49,10,3,4,2,2,2,5,NaN,"[0, 1]"
5,1325,1424,53,10,3,4,2,2,2,5,NaN,NaN
6,1900,1974,76,10,3,4,2,2,2,5,NaN,NaN
7,2625,2849,105,10,3,4,2,2,2,5,"[0, 1]",NaN
8,3200,3274,128,10,3,4,2,2,2,5,NaN,NaN


In [86]:
df_info.to_csv('/content/drive/MyDrive/Project/Object_tracking/scan1_infos2display.csv')